In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bbc-dataset/bbc_data.csv


In [4]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from gensim.models import Word2Vec
import gensim.downloader as api




# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

# Read File
data = pd.read_csv(r'/kaggle/input/bbc-dataset/bbc_data.csv')
df = pd.DataFrame(data)

# Display the first few rows of the DataFrame
print(df.head())

# Convert to lowercase 
df['data'] = df['data'].str.lower()

# Tokenization
df['tokens'] = df['data'].apply(word_tokenize)

# Lemmatization
lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Word2Vec
word2vec_model = Word2Vec(sentences=df['lemmatized'].tolist(), vector_size=100, window=5, min_count=1, workers=4)

# GloVe
glove_model = api.load("glove-wiki-gigaword-100")  # Load GloVe model

# Example usage of GloVe
word_vector = glove_model['example']  # Replace 'example' with any word you want

# Display results
print("TF-IDF DataFrame:")
print(tfidf_df.head())
print("\nWord2Vec example vector for 'example':")
print(word2vec_model.wv['example'])  # Replace 'example' with any word you want
print("\nGloVe vector for 'example':")
print(word_vector)  # Replace 'example' with any word you want

#Visualization using SVM CNN KNN RNN and plot confusion matrrix too

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
                                                data         labels
0  Musicians to tackle US red tape  Musicians gro...  entertainment
1  U2s desire to be number one  U2, who have won ...  entertainment
2  Rocker Doherty in on-stage fight  Rock singer ...  entertainment
3  Snicket tops US box office chart  The film ada...  entertainment
4  Oceans Twelve raids box office  Oceans Twelve,...  entertainment


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [ ]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

# Read File
data = pd.read_csv(r'/kaggle/input/bbc-dataset/bbc_data.csv')
df = pd.DataFrame(data)

# Display the first few rows of the DataFrame
print(df.head())

In [ ]:

# Convert to lowercase 
df['data'] = df['data'].str.lower()

# Tokenization
df['tokens'] = df['data'].apply(word_tokenize)

# Lemmatization
lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [ ]:

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['data'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Prepare labels
labels = df['category']  # Assuming 'category' is the label column

# Split the data
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, test_size=0.2, random_state=42)


In [ ]:

# SVM Model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

# KNN Model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
knn_predictions = knn_model.predict(X_test)

# RNN Model
# Prepare data for RNN
max_words = 1000
embedding_dim = 100


In [ ]:

# Tokenize and pad sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['data'])
X_seq = tokenizer.texts_to_sequences(df['data'])
X_pad = tf.keras.preprocessing.sequence.pad_sequences(X_seq)

# Split the padded sequences
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_pad, labels, test_size=0.2, random_state=42)

# Build RNN model
rnn_model = Sequential()
rnn_model.add(Embedding(max_words, embedding_dim, input_length=X_pad.shape[1]))
rnn_model.add(SpatialDropout1D(0.2))
rnn_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
rnn_model.add(Dense(len(labels.unique()), activation='softmax'))

rnn_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.fit(X_train_rnn, y_train_rnn, epochs=5, batch_size=64, verbose=2)

In [ ]:

# Evaluate models
print("SVM Classification Report:")
print(classification_report(y_test, svm_predictions))

print("KNN Classification Report:")
print(classification_report(y_test, knn_predictions))

# RNN Predictions
rnn_predictions = np.argmax(rnn_model.predict(X_test_rnn), axis=-1)

# Confusion Matrix for SVM
cm_svm = confusion_matrix(y_test, svm_predictions)
disp_svm = ConfusionMatrixDisplay(confusion_matrix=cm_svm, display_labels=labels.unique())
disp_svm.plot()
plt.title('Confusion Matrix for SVM')
plt.show()

# Confusion Matrix for KNN
cm_knn = confusion_matrix(y_test, knn_predictions)
disp_knn = ConfusionMatrixDisplay(confusion_matrix=cm_knn, display_labels=labels.unique())
disp_knn.plot()
plt.title('Confusion Matrix for KNN')
plt.show()

# Confusion Matrix for RNN
cm_rnn = confusion_matrix(y_test_rnn, rnn_predictions)
disp_rnn = ConfusionMatrixDisplay(confusion_matrix=cm_rnn, display_labels=labels.unique())
disp_rnn.plot()
plt.title('Confusion Matrix for RNN')
plt.show()